In [2]:
import polars as pl
import json
from tqdm import tqdm
import numpy as np
import sys

/home/ubuntu/thesis-da/lib/python3.10/site-packages/polars/_cpu_check.py:239: RuntimeWarning: Missing required CPU features.

The following required CPU features were not detected:
    ssse3, sse4.1, sse4.2, popcnt
Continuing to use this version of Polars on this processor will likely result in a crash.
Install the `polars-lts-cpu` package instead of `polars` to run Polars with better compatibility.

Hint: If you are on an Apple ARM machine (e.g. M1) this is likely due to running Python under Rosetta.
It is recommended to install a native version of Python that does not run under Rosetta x86-64 emulation.

If you believe this warning to be a false positive, you can set the `POLARS_SKIP_CPU_CHECK` environment variable to bypass this check.

  warnings.warn(


In [3]:
np.set_printoptions(threshold=sys.maxsize)

In [4]:
to_maintain = [
    "userAgent",
    "eventType",
    "eventName",
    "eventSource",
    "recipientAccountId",
    "awsRegion",
    "eventVersion",
    #"errorCode",
]

```log
081109 203615 148 INFO dfs.DataNode$PacketResponder: PacketResponder 1 for block blk_38865049064139660 terminating
081109 203807 222 INFO dfs.DataNode$PacketResponder: PacketResponder 0 for block blk_-6952295868487656571 terminating
081109 204005 35 INFO dfs.FSNamesystem: BLOCK* NameSystem.addStoredBlock: blockMap updated: 10.251.73.220:50010 is added to blk_7128370237687728475 size 67108864
```

In [5]:
# LineId,Date,Time,Pid,Level,Component,Content,EventId,EventTemplate
# 1,081109,203518,143,INFO,dfs.DataNode$DataXceiver,Receiving block blk_-1608999687919862906 src: /10.250.19.102:54106 dest: /10.250.19.102:50010,E5,Receiving block <*> src: /<*> dest: /<*>
def transtlate_to_syslog(log):
    datetime = log['eventTime']
    userAgent = log['userAgent']
    accountId = log['recipientAccountId']
    source = log['eventSource']
    eventName = log['eventName']
    eventid = log['eventID']
    return f"{datetime}: {eventid} the {accountId} with {userAgent} in the {source} have done {eventName}"    

In [6]:
OUTFILE_HDFS = "../data/prepared/kube_copy_hdfs.log"

In [7]:
outfile = open(OUTFILE_HDFS, "w")

with open('../data/raw/flaws_cloudtrail00.ndjson', "r") as fj:
    for line in tqdm(fj.readlines()):
        jo = json.loads(line)
        outfile.write(f"{transtlate_to_syslog(jo)}\n")
outfile.close()

100%|██████████| 100000/100000 [00:06<00:00, 16147.58it/s]


In [8]:
# Testing drain
%pip install drain3

Note: you may need to restart the kernel to use updated packages.


In [9]:
from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig
import time

In [10]:
template_miner = TemplateMiner()

line_count = 0

with open(OUTFILE_HDFS) as f:
    lines = f.readlines()

print(f"Line of the file: {len(lines)}")

start_time = time.time()
batch_start_time = start_time
batch_size = 100000

for line in lines:
    line = line.rstrip()
    line = line.partition(": ")[2]
    result = template_miner.add_log_message(line)
    line_count += 1
    if line_count % batch_size == 0:
        time_took = time.time() - batch_start_time
        rate = batch_size / time_took
        print(f"Processing line: {line_count}, rate {rate:.1f} lines/sec, "
                    f"{len(template_miner.drain.clusters)} clusters so far.")
        batch_start_time = time.time()
    if result["change_type"] != "none":
        result_json = json.dumps(result)
        print(f"Input ({line_count}): {line}")
        print(f"Result: {result_json}")

time_took = time.time() - start_time
rate = line_count / time_took
print(f"--- Done processing file in {time_took:.2f} sec. Total of {line_count} lines, rate {rate:.1f} lines/sec, "
            f"{len(template_miner.drain.clusters)} clusters")

sorted_clusters = sorted(template_miner.drain.clusters, key=lambda it: it.size, reverse=True)
for cluster in sorted_clusters:
    print(cluster)

print("Prefix Tree:")
template_miner.drain.print_tree()

template_miner.profiler.report(0)

config file not found: drain3.ini


Line of the file: 100000
Input (1): 3038ebd2-c98a-4c65-9b6e-e22506292313 the 811596193553 with [S3Console/0.4] in the s3.amazonaws.com have done ListBuckets
Result: {"change_type": "cluster_created", "cluster_id": 1, "cluster_size": 1, "template_mined": "3038ebd2-c98a-4c65-9b6e-e22506292313 the 811596193553 with [S3Console/0.4] in the s3.amazonaws.com have done ListBuckets", "cluster_count": 1}
Input (2): 22a0d9b1-deea-4d39-827b-2af7050ed3f3 the 811596193553 with console.amazonaws.com in the iam.amazonaws.com have done GetAccountPasswordPolicy
Result: {"change_type": "cluster_template_changed", "cluster_id": 1, "cluster_size": 2, "template_mined": "<*> the 811596193553 with <*> in the <*> have done <*>", "cluster_count": 1}
Input (13): 2dd9d57f-2734-49e4-a801-ab9a7fb21dd3 the 811596193553 with [AWSCloudTrail, aws-internal/3] in the s3.amazonaws.com have done ListBuckets
Result: {"change_type": "cluster_created", "cluster_id": 2, "cluster_size": 1, "template_mined": "2dd9d57f-2734-49e4-

Input (1389): c5472feb-40d2-41c6-9c47-fb15aad0f326 the 811596193553 with Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 in the signin.amazonaws.com have done ConsoleLogin
Result: {"change_type": "cluster_created", "cluster_id": 4, "cluster_size": 1, "template_mined": "c5472feb-40d2-41c6-9c47-fb15aad0f326 the 811596193553 with Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36 in the signin.amazonaws.com have done ConsoleLogin", "cluster_count": 4}
Input (1592): 4d7c0add-9e21-4ed9-8238-65055cb87b7c the 811596193553 with [Cyberduck/4.7 (459548).459548 (Windows 8/6.2) (x86)] in the s3.amazonaws.com have done GetBucketVersioning
Result: {"change_type": "cluster_created", "cluster_id": 5, "cluster_size": 1, "template_mined": "4d7c0add-9e21-4ed9-8238-65055cb87b7c the 811596193553 with [Cyberduck/4.7 (459548).459548 (Windows 8/6.2) (x86)] in the s3.amazonaws.com have done Ge

In [11]:
df = pl.read_csv("../data/prepared/tokenized_data_dot_product.csv")

In [12]:
df.head()

userAgent-token,eventType-token,eventName-token,eventSource-token,recipientAccountId-token,awsRegion-token,eventVersion-token,dot
i64,i64,i64,i64,i64,i64,i64,f64
3827,2,594,100,0,13,2,4538.0
3827,2,594,100,0,3,2,4528.0
3827,2,594,100,0,3,2,4528.0
3827,2,594,100,0,8,2,4533.0
3827,2,594,100,0,8,2,4533.0


In [13]:
df.select(pl.col("*")).unique()

userAgent-token,eventType-token,eventName-token,eventSource-token,recipientAccountId-token,awsRegion-token,eventVersion-token,dot
i64,i64,i64,i64,i64,i64,i64,f64
3827,2,298,100,0,11,2,4240.0
3827,2,828,100,0,12,2,4771.0
3827,2,298,100,0,6,2,4235.0
3827,2,1236,100,0,14,2,5181.0
1792,2,533,142,0,14,2,2485.0
4009,2,594,100,0,8,2,4715.0
4009,2,594,100,0,11,2,4718.0
4009,2,594,100,0,16,2,4723.0
4009,2,1236,100,0,1,2,5350.0


In [14]:
df.select(pl.col("dot").value_counts()).unnest("dot").select(pl.col("*").sort_by("count", descending=True))

dot,count
f64,u32
4710.0,69110
4715.0,69035
4719.0,60698
4720.0,59486
4533.0,57086
4534.0,54661
4716.0,51946
4528.0,51641
4712.0,49843


In [15]:
# 4710.0

df.filter(pl.col("dot") == 4710.0).unique()

userAgent-token,eventType-token,eventName-token,eventSource-token,recipientAccountId-token,awsRegion-token,eventVersion-token,dot
i64,i64,i64,i64,i64,i64,i64,f64
4516,2,136,48,0,5,3,4710.0
3742,2,842,108,0,14,2,4710.0
3464,2,1086,142,0,14,2,4710.0
4498,2,185,8,0,15,2,4710.0
4516,2,91,86,0,13,2,4710.0
4526,2,43,132,0,5,2,4710.0
4516,2,43,132,0,15,2,4710.0
4009,2,594,100,0,3,2,4710.0
3432,2,1159,100,0,15,2,4710.0


In [16]:
df.select(pl.col("dot").value_counts()).unnest("dot").filter(pl.col("count") == 1).count()["dot"][0]

87

In [17]:
dim = df.shape
correct = df.select(pl.col("dot").value_counts()).unnest("dot").filter(pl.col("count") == 1).count()["dot"][0]
incorrect = df.select(pl.col("dot").value_counts()).unnest("dot").filter(pl.col("count") != 1).count()["dot"][0]
print(f"Correct-ness: {correct/dim[0]}")
print(f"INcorrect-ness: {incorrect/dim[0]}")

Correct-ness: 4.48637225751158e-05
INcorrect-ness: 0.0050587714765734015


In [18]:
from sklearn.cluster import KMeans

In [19]:
dlp = pl.read_csv("../data/prepared/tokenized_data.csv")
dlp.head()

userAgent-token,eventType-token,eventName-token,eventSource-token,recipientAccountId-token,awsRegion-token,eventVersion-token
i64,i64,i64,i64,i64,i64,i64
3827,2,594,100,0,13,2
3827,2,594,100,0,3,2
3827,2,594,100,0,3,2
3827,2,594,100,0,8,2
3827,2,594,100,0,8,2


In [20]:
dlp.shape[1]

7

In [21]:
X_train = dlp.to_numpy()

kmeans = KMeans(n_clusters=dlp.shape[1], random_state=0, n_init="auto").fit(X_train)

In [22]:
kmeans.predict(dlp[134])

/home/ubuntu/thesis-da/lib/python3.10/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but KMeans was fitted without feature names
  warnings.warn(


array([0], dtype=int32)

In [23]:
dlp[134], dlp[0]

(shape: (1, 7)
 ┌──────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┬─────────────┐
 │ userAgent-to ┆ eventType-t ┆ eventName-t ┆ eventSource ┆ recipientAc ┆ awsRegion-t ┆ eventVersio │
 │ ken          ┆ oken        ┆ oken        ┆ -token      ┆ countId-tok ┆ oken        ┆ n-token     │
 │ ---          ┆ ---         ┆ ---         ┆ ---         ┆ en          ┆ ---         ┆ ---         │
 │ i64          ┆ i64         ┆ i64         ┆ i64         ┆ ---         ┆ i64         ┆ i64         │
 │              ┆             ┆             ┆             ┆ i64         ┆             ┆             │
 ╞══════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╪═════════════╡
 │ 3827         ┆ 2           ┆ 594         ┆ 100         ┆ 0           ┆ 3           ┆ 2           │
 └──────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┴─────────────┘,
 shape: (1, 7)
 ┌──────────────┬─────────────┬─────────────┬──────

# To test:
- Knearest fatto bene
- drain3 con `requestResponse`/`requestObject` messo a testo

In [24]:
from sklearn.neighbors import NearestNeighbors

In [26]:
nbrs = NearestNeighbors(n_neighbors=8, algorithm='ball_tree').fit(X_train)

In [28]:
distances, indices = nbrs.kneighbors(X_train)

In [1]:
indices

NameError: name 'indices' is not defined

In [ ]:
distances

In [ ]:
import seaborn as sns
import matplotlib as mlp
import matplotlib.pyplot as plt

In [ ]:
sns.scatterplot(indices)

In [ ]:
sns.scatterplot(distances)